This notebook is ran in a docker container where the project directory (i.e. same directory as README.md) is located in `/code`, which is set below. If you run locally you'll need to set the path of your project directory accordingly.

In [1]:
%cd /code

/code


---

The `load_dotenv` function below loads all the variables found in the `.env` file as environment variables. You must have a `.env` file located in the project directory containing your OpenAI API key, in the following format.

```
OPENAI_API_KEY=sk-...
```

In [2]:
from dotenv import load_dotenv
from notebook_helpers import usage_string, mprint

load_dotenv()

True

---

# workflows

A `workflow` is an object that executes a sequence of tasks. Each task in the workflow is a callable, which can be either a function or an object that implements the `__call__` method. **The output of one task serves as the input to the next task in the workflow. So a `workflow` is a simple mechanism that takes an input and sends the input to the first task, and the propegates the output of the first task to the second task, and so on, until the end of the workflow is reached, and returns the final result.** 

Furthermore, a workflow aggregates the history (prompts/responses, token usage, costs, etc.) across all tasks. More specifically, it aggregates all of the `Record` objects across any task that has a `history` method (which returns a list of Record objects; a Record object contains the metadata of an event such as costs, tokens used, prompt, response, etc.). This functionality allows the workflow to contain convenient methods that aggregate the costs and usage across all tasks of the workflow, and can also be used to explore intermediate steps and events in the workflow.

---

## Example - Using a model as a prompt enhancer

Here's an example of a simple "prompt enhancer", where the user provides a prompt, and the first model enhances the user's prompt, then second model provides a response based on the enhanced prompt.

- first task: defines a prompt-template that takes the user's prompt, and creates a new prompt asking a chat model to improve the prompt (within the context of creating python code)
- second task: the `prompt_enhancer` model takes the modified prompt and improves the prompt
- third task: the `chat_assistant` model takes the response from the last model (which is an improved prompt) and returns the request
- fourth task: ignores the response from the chat model; creates a new prompt asking the chat model to extract the relevant code created in the previous response
- fifth task: the chat model, which internally maintains the history of messages, returns only the relevant code from the previous response.

In [3]:
from llm_workflow.base import Workflow
from llm_workflow.openai import OpenAIChat

prompt_enhancer = OpenAIChat()
# different model/object, therefore different message history (i.e. conversation)
chat_assistant = OpenAIChat()

def prompt_template(user_prompt: str) -> str:
    return "Improve the user's request, below, by expanding the request " \
        "to describe the relevant python best practices and documentation " \
        f"requirements that should be followed:\n\n```{user_prompt}```"

def prompt_extract_code(_) -> str:
    # `_` signals that we are ignoring the input (from the previous task)
    return "Return only the primary code of interest from the previous answer, "\
        "including docstrings, but without any text/response."

# The only requirement for the list of tasks is that each item/task is a
# callable where the output of one task matches the input of the next task.
# The input to the workflow is passed to the first task;
# the output of the last task is returned by the workflow.
workflow = Workflow(tasks=[
    prompt_template,      # modifies the user's prompt
    prompt_enhancer,      # returns an improved version of the user's prompt
    chat_assistant,       # returns the chat response based on the improved prompt
    prompt_extract_code,  # prompt to ask the model to extract only the relevant code
    chat_assistant,       # returns only the function from the model's last response
])
prompt = "create a function to mask all emails from a string value"
response = workflow(prompt)
print(response)

```python
import re

def mask_emails(input_string: str) -> str:
    """
    Mask all emails from the input string.

    Args:
    input_string (str): The input string containing emails to be masked.

    Returns:
    str: The input string with emails masked.

    This function uses regular expressions to find and mask all email addresses in the input string.
    """

    # Regular expression pattern to match email addresses
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'

    # Find all email addresses in the input string
    emails = re.findall(email_pattern, input_string)

    # Mask each email address with '***masked***'
    for email in emails:
        input_string = input_string.replace(email, '***masked***')

    return input_string
```


In [4]:
print(f"Cost:            ${workflow.sum('cost'):.4f}")
print(f"Total Tokens:     {workflow.sum('total_tokens'):,}")
print(f"Prompt Tokens:    {workflow.sum('input_tokens'):,}")
print(f"Response Tokens:  {workflow.sum('response_tokens'):,}")

Cost:            $0.0028
Total Tokens:     1,942
Prompt Tokens:    1,104
Response Tokens:  838


We can view the history of the workflow (i.e. the aggregated history across all tasks) with the `workflow.history()` method. 

In this example, the only class that tracks history is `OpenAIChat`. Therefore, both the `prompt_enhancer` and `chat_assistant` objects will contain history. `workflow.history()` will return a list of three `ExchangeRecord` objects. The first record corresponds to our request to the `prompt_enhancer`, and the second two records correspond to our `chat_assistant` requests. An ExchangeRecord represents a single exchange/transaction with an LLM, encompassing an input (`prompt`) and its corresponding output (`response`), along with other properties like `cost` and `token_tokens`.

In [5]:
[type(x) for x in workflow.history()]

[llm_workflow.base.ExchangeRecord,
 llm_workflow.base.ExchangeRecord,
 llm_workflow.base.ExchangeRecord]

We can view the response we received from the `prompt_enhancer` model by looking at the first record's `response` property (or the second record's `prompt` property since the workflow passes the output of `prompt_enhancer` as the input to the `chat_assistant`):

In [6]:
mprint(workflow.history()[0].response)

To create a function to mask all emails from a string value in Python, it's important to follow best practices and document the code properly. Here are some best practices and documentation requirements to consider:

1. Function Name and Parameters:
   - Choose a descriptive and meaningful name for the function, such as `mask_emails`.
   - Clearly define the input parameter(s) of the function, such as the string value containing emails.

2. Docstring:
   - Include a docstring at the beginning of the function to describe its purpose, input parameters, and return value.
   - Use descriptive language to explain how the function works and what it does.

3. Comments:
   - Add comments within the function to explain complex logic or any non-obvious code.

4. Error Handling:
   - Consider adding error handling to handle cases where the input string does not contain a valid email format.

5. Testing:
   - Write test cases to ensure the function works as expected and handles different scenarios, such as multiple emails in the input string, emails with different formats, and edge cases.

6. PEP 8 Compliance:
   - Follow the PEP 8 style guide for Python code, including proper indentation, naming conventions, and code layout.

7. Version Control and Documentation:
   - If the function is part of a larger project, consider using version control (e.g., Git) and documenting the function in the project's documentation.

By following these best practices and documentation requirements, you can ensure that the function to mask emails is well-documented, maintainable, and easy to understand for other developers who may use or contribute to the code.

We could also view the original response from the `chat_assistant` model:

In [7]:
mprint(workflow.history()[1].response)

Here's an example of how you can create a function to mask emails from a string value in Python while following the best practices and documentation requirements you've outlined:

```python
import re

def mask_emails(input_string: str) -> str:
    """
    Mask all emails from the input string.

    Args:
    input_string (str): The input string containing emails to be masked.

    Returns:
    str: The input string with emails masked.

    This function uses regular expressions to find and mask all email addresses in the input string.
    """

    # Regular expression pattern to match email addresses
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'

    # Find all email addresses in the input string
    emails = re.findall(email_pattern, input_string)

    # Mask each email address with '***masked***'
    for email in emails:
        input_string = input_string.replace(email, '***masked***')

    return input_string

# Test cases
test_string = "Please contact me at user@example.com for further information."
masked_string = mask_emails(test_string)
print(masked_string)  # Output: "Please contact me at ***masked*** for further information."
```

By following these best practices and documentation requirements, the function is well-documented, maintainable, and easy to understand for other developers. It also includes error handling, PEP 8 compliance, and a simple test case to ensure the function works as expected.

The final response returned by the `chat_assistant` (and by the `workflow` object) returns only the `mask_email_addresses` function. The `response` object should match the `response` value in the last record (`workflow.history()[-1].response`).

In [8]:
assert response == workflow.history()[-1].response

---

## Example -- Using the results of a web-search in a chat message.

In the following example, we will ask a chat model a question, and provide the model with additional context based on the most relevant text we find in a web-search. For the sake of the example, we will ask the chat model to the answer it provides and summarize that answer.  The workflow is defined as follows:


|   input   |  output  |   task description   |
|-----------|----------|----------|
|  `None` |   str    |  Ask a question: e.g. `"What is the meaning of life?"`  |
|    str    |   urls   |  Do a web-search via `DuckDuckGo` based on the question.  |
|   urls    | Documents|  Take the urls, scrape the corresponding web-pages, and convert to a list of `Document` objects  |
| Documents | Documents|  Split the `Document` objects into smaller chunks (also `Document` objects)  |
| Documents | `None` |  Store the Documents in a `Chroma` document index so that we can lookup the most relevant documents |
| `None`  |   str    |  Return the original question `What is the meaning of life?` |
|    str    |   str    |  Take the original question, lookup the most relevant documents, and construct the corresponding prompt with the documents injected into the prompt. |
|    str    |   str    |  Pass the prompt to the chat model; receive a response |
|    str    |   str    |  Construct a new prompt, containing the response and a request to summarize the response. |
|    str    |   str    |  Pass the new prompt to the chat model; receive a response |

**Notice how the output of one task matches the input of the next task. This means you could swap out any of the objects we use below with your own custom functions or classes and the only requirement is that the input/output matches for that particular task.**

### Initial objects in workflow

Let's define the objects and functions we are going to include in the workflow. We're including many tasks, so that we can see how flexible our workflow can be.

Refer to the notebooks in https://github.com/shane-kercheval/llm-workflow/tree/main/examples for examples of how the various helper classes below (e.g. `DuckDuckGoSearch`, `scrape_url`, etc.) are used.

#### Quick note on the `Value` object.

The only bit of magic below is that we're using a `Value` object to cache the initial question, feed it into the web-search, and then inject it back into the workflow at a later point (passing it to the prompt-template). A `Value` object is a callable that, when called with a value caches and returns the same value, and when called without a value, simply returns the previously cached value. When you understand what it's doing, it's not magic at all. Here's a simple example:

In [9]:
from llm_workflow.base import Value
cache = Value()
result = cache("This is a value")  # calling the object with a value caches and returns the value
print(result)  # the value in `result` is whatever was passed in to the Value object
print(cache())  # calling the object without a parameter returns the cached value
result = cache("new value")
print(result)
print(cache())

This is a value
This is a value
new value
new value


In [18]:
import re
from llm_workflow.base import Document, Workflow, Value
from llm_workflow.openai import OpenAIEmbedding, OpenAIChat
from llm_workflow.utilities import DuckDuckGoSearch, scrape_url, split_documents
from llm_workflow.indexes import ChromaDocumentIndex
from llm_workflow.prompt_templates import DocSearchTemplate

# Seach DuckDuckGo based on initial question (passed into the workflow)
duckduckgo_search = DuckDuckGoSearch(top_n=3)

# define a function that takes the tasks from the web-search, scrapes the web-pages,
# and then creates Document objects from the text of each web-page
def scrape_urls(search_results):
    """
    For each url (i.e. `href` in `search_results`):
    - extracts text
    - replace new-lines with spaces
    - create a Document object
    """
    def _scrape_url(url):
        try:
            return scrape_url(url)
        except:
            return ''
        
    return [
        Document(content=re.sub(r'\s+', ' ', _scrape_url(x['href'])))
        for x in search_results
    ]

# Embeddings model used for document index/search (the documents created from the web-search)
embeddings_model = OpenAIEmbedding()
document_index = ChromaDocumentIndex(embeddings_model=embeddings_model, n_results=3)
# DocSearchTemplate uses the ChromaDocumentIndex object to search for the most relevant documents
# (from the web-search) based on the intitial question (which it takes as an input)
prompt_template = DocSearchTemplate(doc_index=document_index, n_docs=3)
# Create a chat model without streaming.
non_streaming_chat = OpenAIChat()
# Create a chat model with streaming enabled via callback.
streaming_chat = OpenAIChat(streaming_callback=lambda x: print(x.response, end='|'))

# A `Value` object is a simple caching mechanism. It's a callable that, when passed a value, it
# caches and returns that value; and when called without a value, it returns the cached value.
# Below, it's being used to cache the original question, feed the question into the web-search
# (`DuckDuckGoSearch`), and then re-inject the question back in the workflow and into the
# prompt-template (`DocSearchTemplate`).
question = Value()
# This simple function takes the response from the original chat model and creates a prompt that
# asks the model to summarize the response.
question_2 = lambda x: f'Summarize the following in less than 20 words: "{x}"'

### Defining and Running the workflow

In [17]:
# each task is a callable where the output of one task is the input to the next
workflow = Workflow(tasks=[
    question,
    duckduckgo_search,
    scrape_urls,
    split_documents,  # split web-pages into smaller chunks; defaults to chunk-size of 500
    document_index,  # __call__ function calls add() if given a list of documents (which is returned by `split_documents`)
    question,
    prompt_template,
    non_streaming_chat,
    question_2,
    streaming_chat,
])
# the value passed into `workflow()` is passed to the `initial_question` object (which
# is `Value` object and so it caches the value passed in and also returns it) which then gets
# passed to the `duckduckgo_search` object, and so on.
# the response of the final model is streamed, because our chat model has the streaming_callback
# set, but it should also match the response returned
response = workflow("What is langchain?")

Lang|Chain| is| an| open|-source| framework| for| developing| applications| using| large| language| models|,| offering| standardized| interfaces| for| various| tasks|.|

In [20]:
mprint("> " + response)

> LangChain is an open-source framework for developing applications using large language models, offering standardized interfaces for various tasks.

---

### Total costs and usage

The `workflow` object aggregates the costs/usage across all tasks that have a `history` property where that `history` property returns `UsageRecord` objects. In other words, any task that tracks its own history automatically gets counted towards the total usage within the workflow.

In [21]:
# usage_string simply uses the properties in workflow (workflow.sum('cost'), workflow.sum('total_tokens'), etc.)
# to create a formatted string
print(usage_string(workflow))

Cost:              $0.00134
Total Tokens:       7,587
Prompt Tokens:      443
Response Tokens:    98
Embedding Tokens:   7,046



In [22]:
# to get the number of embedding tokens, sum `total_tokens` across only EmbeddingRecord objects
from llm_workflow.base import EmbeddingRecord
workflow.sum('total_tokens', types=EmbeddingRecord)

7046

### History

Similar to tracking the costs/usage, we can dig into the history at a more granular level.

As you can see below, the `history` property returns a list containing:

- a `SearchRecord` object capturing our original search query and results
- two `EmbeddingsRecord` records; the first corresponds to getting the embeddings of all of the chunks from the web-pages; the second corresponds to getting the embedding of our original question so that we can find the most relevant chunks; the `EmbeddingsRecord` is a `UsageRecord` so it contains costs/usage
- two `MessageRecord` records; the first corresponds to the first question (prompt & response) we made to the chat model; the second corresponds to our second query to the chat model asking it to summarize the first response; the `MessageRecord` is a `UsageRecord` so it contains costs/usage

In [23]:
[type(x) for x in workflow.history()]

[llm_workflow.base.SearchRecord,
 llm_workflow.base.EmbeddingRecord,
 llm_workflow.base.EmbeddingRecord,
 llm_workflow.base.ExchangeRecord,
 llm_workflow.base.ExchangeRecord]

In [24]:
print(workflow.history()[3])
mprint('---')
print(workflow.history()[4])

timestamp: 2023-11-09 04:10:16.679; prompt: "Answer the question at the end of the text as trut..."; response: "LangChain is an open-source framework that aims to...";  cost: $0.000489; total_tokens: 413; input_tokens: 337; response_tokens: 76; uuid: 659fb817-9484-4def-91cf-4ba0080f82ea


---

timestamp: 2023-11-09 04:10:17.623; prompt: "Summarize the following in less than 20 words: "La..."; response: "LangChain is an open-source framework for developi...";  cost: $0.000150; total_tokens: 128; input_tokens: 106; response_tokens: 22; uuid: 8759a4fd-dda9-40b6-9184-658b186d94cc


#### `exchange_history`

We can use the `exchange_history` property which simply returns all of the history items that are of type `ExchangeRecord`

In [25]:
from llm_workflow.base import ExchangeRecord
len(workflow.history(ExchangeRecord))

2

In [26]:
print(workflow.history(ExchangeRecord)[0])  # same as workflow.history()[3]
mprint('---')
print(workflow.history(ExchangeRecord)[1])  # same as workflow.history()[4]

timestamp: 2023-11-09 04:10:16.679; prompt: "Answer the question at the end of the text as trut..."; response: "LangChain is an open-source framework that aims to...";  cost: $0.000489; total_tokens: 413; input_tokens: 337; response_tokens: 76; uuid: 659fb817-9484-4def-91cf-4ba0080f82ea


---

timestamp: 2023-11-09 04:10:17.623; prompt: "Summarize the following in less than 20 words: "La..."; response: "LangChain is an open-source framework for developi...";  cost: $0.000150; total_tokens: 128; input_tokens: 106; response_tokens: 22; uuid: 8759a4fd-dda9-40b6-9184-658b186d94cc


#### First prompt to the chat model

You can see below that we used three chunks from our original web-search in the first prompt we sent to ChatGPT.

In [27]:
mprint(workflow.history(ExchangeRecord)[0].prompt)


Answer the question at the end of the text as truthfully and accurately as possible, based on the following information provided.

Here is the information:

```
Demystifying LangChain: A Brief Introduction and Implementation Overview / Blogs / Perficient Skip to main content Generative AI Demystifying LangChain: A Brief Introduction and Implementation Overview by Shubham Jaiswal on November 8th, 2023 | ~ minute read Introduction LangChain represents an open-source framework that aims to streamline the development of applications leveraging large language models (LLMs). Its primary objective is to establish a standardized interface for chains, boasting

computing, machine learning and artificial intelligence, digital marketing, cybersecurity, data analysis, and project management. Conclusion: That was the basic introduction to langchain framework. I hope you have understood the usage and there are a lot more concepts such as prompt templates, chains and agents to learn. The LangChain framework is a great interface to develop interesting AI-powered applications and from personal assistants to prompt management as well as automating tasks. So,

classification: LangChain can be used for text classifications and sentiment analysis with the text input data Text summarization: LangChain can be used to summarize the text in the specified number of words or sentences. Machine translation: LangChain can be used to translate the input text data into different languages. LangChain Key Concepts: The main properties of LangChain Framework are : Components: Components are modular building blocks that are ready and easy to use to build powerful
```

Here is the question:

```
What is langchain?
```


#### First response from the chat model

In [28]:
mprint("> " + workflow.history(ExchangeRecord)[0].response)

> LangChain is an open-source framework that aims to streamline the development of applications leveraging large language models (LLMs). Its primary objective is to establish a standardized interface for chains, boasting computing, machine learning, artificial intelligence, digital marketing, cybersecurity, data analysis, and project management. LangChain can be used for text classifications, sentiment analysis, text summarization, and machine translation.

#### Second prompt to the chat model

In [29]:
mprint(workflow.history(ExchangeRecord)[1].prompt)

Summarize the following in less than 20 words: "LangChain is an open-source framework that aims to streamline the development of applications leveraging large language models (LLMs). Its primary objective is to establish a standardized interface for chains, boasting computing, machine learning, artificial intelligence, digital marketing, cybersecurity, data analysis, and project management. LangChain can be used for text classifications, sentiment analysis, text summarization, and machine translation."

#### Second response from the chat model

In [30]:
mprint("> " + workflow.history(ExchangeRecord)[1].response)

> LangChain is an open-source framework for developing applications using large language models, offering standardized interfaces for various tasks.

---